# Transformermodellen for prediksjon av sekvenser

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

OPPGAVE 1.1

For å trene en transformermodell så trenger vi et datasett som består at flere x og y vektorer, $ D = \{ (x_0, y_0), (x_1, y_1), \ldots, (x_{n-1}, y_{n-1}) \} $. For å vise et eksempel på et av punktene i datasettet, $(x_0, y_0)$, velger vi $r = 2$, $a = 43$, $b = 7$, $c = 18$ og $d = a \cdot b + c = 319$. Vektoren $x_0$ er bygget opp slik at vi får de $r$ første sifferene av $a$, $b$, $c$ og $d$ og lengden til $x_i$ blir dermed $4 \cdot r$. I dette tilfellet får vi da $x_0 = [4, 3, 0, 7, 8, 3, 1]$. $y_0 $ består av alle sifferene til $d$ og blir her $y_0 = [3, 1, 9]$.  




OPPGAVE 1.2


Vi skal  nå anta at optimeringen er ferdig og at vi har funnet parametrene ${\theta}$ som minimerer objektfunksjonen $\mathscr{L}$, vi skal dermed bruke modellen $f_{\theta}$ til å predikere $d$ gitt $a, b$ og $c$. Modellen $f_{\theta}$ skal testes på en sekvens som bare inneholder tallene til $a, b,$ og $c$ og vi befinner oss i testfasen. I testfasen skal $x\in \mathbb{Z^{2r}}$, ved å kjøre $x$ gjennom modellen skal vi få ut som gir ut $f_{\theta}(x) = \hat{z}$ der $\hat{z}\in \mathbb{Z^{2r}}$, altså vi skal få ut en sekvens av samme dimensjon som sekvensen som puttes inn.

Vi har fortsatt $r = 2$, $a = 43$, $b = 7$ og $c = 18$ og skal predikere $d$. For å predikere $d\in \mathbb{Z^{r+1}}$ skal hver evaluering av modellen predikerer neste sifferet i sekvensen  og legger det til bakerst i $x$, dette blir input neste prediksjon helt til det er predikert $r+1$ nye heltall som ideelt sett skal være lik $d$. 


Vi ønsker å finne ${\theta}$ slik at $\hat{y} = [\hat{z_5}, \hat{z_6}, \hat{z_7}] = [3, 1, 9]$


Vi starter med første sekvens, $x^{(0)} = [4,3,0,7,1,8]$, og setter inn i modellen $f_{\theta}([4,3,0,7,1,8]) = [\hat{z_0}^{(0)}, \hat{z_1}^{(0)},..., \hat{z_5}^{(0)}]$ 

Legger til bakerste siffer fra outputten inn i neste sekvens: $x^{(1)} = [4,3,0,7,1,8, \hat{z_5}^{(0)}]$. Predikerer neste sekvens: $f_{\theta}([4,3,0,7,1,8, \hat{z_5}^{(0)}]) = [\hat{z_0}^{(1)}, \hat{z_1}^{(1)},..., \hat{z_6}^{(1)}]$ 

Gjentar prosessen: $x^{(2)} = [4,3,0,7,1,8, \hat{z_5}^{(0)}, \hat{z_6}^{(1)}]$, $f_{\theta}([4,3,0,7,1,8, \hat{z_5}^{(0)}], \hat{z_6}^{(1)}) = [\hat{z_0}^{(2)}, \hat{z_1}^{(2)},..., \hat{z_7}^{(2)}]$

Etter $3$ evalueringer av modellen ender vi opp med sekvensen: $x^{(3)} = [4,3,0,7,1,8, \hat{z_5}^{(0)}, \hat{z_6}^{(1)}, \hat{z_7}^{(2)}]$

Det er nå predikert $r + 1 = 3$ nye heltall, $\hat{y} = [\hat{z_5}^{(0)}, \hat{z_6}^{(1)}, \hat{z_7}^{(2)}]$. Nå kan vi sammenligneden predikerte $\hat{y}$ med vår $y_0 = [3, 1, 9]$. Dersom vi antar at modellen vår er godt trent skal dette stemme $\hat{z} = \hat{y} = y = [3,1,9]$.

OPPGAVE 1.3

Anta at vi bruker cross-entropy som objektfunksjon, at $m = 5$ og $y = [4, 3, 2, 1]$.

Hvilke diskret sannsynlighetsfordeling $\hat{Y}$ ville gitt en objektfunksjon $\mathscr{L}({\theta},D) = 0 $?

Hva ville ${\hat{y}}$ vært i dette tilfellet?

Ved å bruke cross-entropy funksjonen $\mathscr{L}({\theta},D)$ som loss funksjon og setter denne lik $0$ antar vi at modellen vår er en perfekt prediksjon. Dersom $\hat{Y} = F_{\theta}(x) = onehot(y)$ får vi nettopp dette få $\mathscr{L}({\theta},D) = 0$. Dette betyr også at $\hat{y} = argmax_{col}(F_{\theta}(x))$

Får oppgitt $m = 5$ og $y = [4, 3, 2, 1]$ får vi:

\begin{equation}

onehot(y) = onehot([4, 3, 2, 1]) := \begin{bmatrix} 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\\end{bmatrix} = \hat{Y}

\end{equation}

Fordelingen til $\hat{Y}$ tilsvarer en coloumnwise onehot distribution. $1$ representerer den korekte indeksen, ellers er det fyllt med $0$. 
Setter inn for $y$ i $argmax$

$\hat{y} = argmax_{col}(F_{\theta}(x)) = argmax_{col}\begin{bmatrix} 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\\end{bmatrix} = [4,3,2,1]$


$F_{\theta}(x)$ tilfredstiller kravene over $m = 5$ utfall og at $\hat{y}$ er en sekvens med lengde $n = 4$ med heltatt opp til $m- 1 = 5 - 1 = 4$



OPPGAVE 1.4

Gitt $d, m, n_{max}, k, p$ og $L$ så skal vi nå beregne hvor mange enkeltparametere, $w \in \mathbb{R}$, vi må bestemme ved optimering. For å finne dette må vi summere opp enkeltparameterene i alle parametermatrisene. Gitt

$\theta = {W_E, W_P, W_U, {{W_O}^{(l)}, {W_V}^{(l)}, {W_Q}^{(l)}, {W_K}^{(l)}, {W_1}^{(l)}, {W_2}^{(l)}}_{l=0}^{L-1}}$

Dimensjonen til $W_E$ og $W_U$ er $d \times m$, dimensjonen til $W_P$ er $d \times n_{max}$ , dimensjonen til ${W_O}^{(l)}, {W_V}^{(l)}, {W_Q}^{(l)}$ og ${W_K}^{(l)}$ er $k \times d$ og dimensjonen til ${W_1}^{(l)}$ og ${W_2}^{(l)}$ er $p \times d$. 

Vi får da totalt $w = 2*d*m + d*n_{max} + (4*k*d + 2*p*d)*L$ enkeltparametere som vi kan optimere.